In [1]:
import uproot
import awkward as ak
import numpy as np
from coffea import nanoevents

In [36]:
pf_variables = [
    "pfcand_pt_log_nopuppi",
    "pfcand_e_log_nopuppi",
    "pfcand_etarel",
    "pfcand_phirel",
    "pfcand_isEl",
    "pfcand_isMu",
    "pfcand_isGamma",
    "pfcand_isChargedHad",
    "pfcand_isNeutralHad",
    "pfcand_abseta",
    "pfcand_charge",
    "pfcand_VTX_ass",
    "pfcand_lostInnerHits",
    "pfcand_normchi2",
    "pfcand_quality",
    "pfcand_dz",
    "pfcand_dzsig",
    "pfcand_dxy",
    "pfcand_dxysig",
    "pfcand_btagEtaRel",
    "pfcand_btagPtRatio",
    "pfcand_btagPParRatio",
    "pfcand_btagSip3dVal",
    "pfcand_btagSip3dSig",
    "pfcand_btagJetDistVal",
]

sv_variables = [
    "sv_pt_log",
    "sv_mass",
    "sv_etarel",
    "sv_phirel",
    "sv_abseta",
    "sv_ntracks",
    "sv_normchi2",
    "sv_dxy",
    "sv_dxysig",
    "sv_d3d",
    "sv_d3dsig",
    "sv_costhetasvpv",
]

In [167]:
events_dnn = uproot.open("output_numEvent40.root:deepntuplizer/tree").arrays()

fname = "nano_skim_0-1.root"
events_coffea = uproot.open(f"{fname}:Events").arrays(library="np")

In [3]:
events_pf = nanoevents.NanoEventsFactory.from_root(
    "miniaod_20ul_35914-0.root",
    schemaclass=nanoevents.NanoAODSchema,
).events()

/Users/raghav/mambaforge/envs/python310/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nBHadrons in <TTree 'Events' (1929 branches) at 0x7fd4a8521d50>, taking first instance
  warnings.warn(
/Users/raghav/mambaforge/envs/python310/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nCHadrons in <TTree 'Events' (1929 branches) at 0x7fd4a8521d50>, taking first instance
  warnings.warn(
/Users/raghav/mambaforge/envs/python310/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nBHadrons in <TTree 'Events' (1929 branches) at 0x7fd4a8521d50>, taking first instance
  warnings.warn(
/Users/raghav/mambaforge/envs/python310/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nCHadrons in <TTree 'Events' (1929 branches) at 0x7f

In [173]:
npfs = 128
nsvs = 10
idx_pfnano = 7
idx_dnn = 20


def pad_val(array, var_length):
    return (
        ak.pad_none(array, var_length, axis=1, clip=True).to_numpy().filled(fill_value=0)
    ).astype(np.float32)


def get_dnn(branch, idx=idx_dnn):
    if "pfcand" in branch:
        x = pad_val(events_dnn[branch], npfs)
        y = x[idx]
        # y = np.delete(y,28)
        # y = np.append(y,0.)
    elif "sv" in branch:
        y = pad_val(events_dnn[branch], nsvs)[idx]
    else:
        y = events_dnn[branch][idx]
    return y


def get_coffea(branch, idx=idx_pfnano):
    x = events_coffea[branch][idx]
    if "pfcand" in branch:
        sorter = np.copy(events_coffea["pfcand_pt_log_nopuppi"][idx])
        sorter[sorter == 0] = -100
        x = x[ak.argsort(sorter, ascending=False)]

    elif "sv" in branch:
        x = x[ak.argsort(events_coffea["sv_dxysig"][idx], ascending=False)]
    
    return x


In [94]:
events_coffea["fj_pt"]

array([327.  , 504.25, 414.  , 503.5 , 469.5 , 499.  , 307.5 , 498.75,
       381.25, 308.5 , 337.5 , 297.  , 395.75, 275.  , 271.75, 313.25,
       376.75])

In [69]:
(events_pf.FatJet.pt * (1.-events_pf.FatJet.rawFactor))[1]

<Array [465, 251] type='2 * float32'>

In [70]:
(events_pf.FatJet.pt)[1]

<Array [504, 275] type='2 * float32[parameters={"__doc__": "pt"}]'>

In [68]:
events_dnn["fj_pt"]

<Array [465, 250] type='2 * float32'>

In [74]:
get_dnn("pfcand_dz")

array([ 1.45874021e-03,  0.00000000e+00, -1.80053711e-03,  0.00000000e+00,
        0.00000000e+00,  7.26562506e-03,  0.00000000e+00,  0.00000000e+00,
       -2.62451172e-03,  0.00000000e+00, -2.68798834e-03, -2.50732433e-03,
       -1.97998038e-03, -5.94238285e-03,  8.64868169e-04, -1.44409179e-03,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  3.90773058e+00,  0.00000000e+00,  1.02533504e-01,
        0.00000000e+00,  1.24511719e-02, -5.28320298e-03,  4.45800787e-03,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [185]:
get_dnn("pfcand_normchi2")

array([  0.,   0., 999.,   1., 999., 999., 999.,   0.,   2., 999., 999.,
         0., 999., 999., 999., 999.,   0.,   1.,   1.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.], dtype=float32)

In [184]:
get_coffea("pfcand_normchi2")

array([ 0.,  0., -1.,  1., -1., -1., -1.,  0.,  2., -1., -1.,  0., -1.,
       -1., -1., -1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
      dtype=float32)

In [ ]:
get_

In [174]:
for var in pf_variables:
    x = get_dnn(var)
    y = get_coffea(var)

    if not np.allclose(x, y, rtol=1e-2):
        not_close = np.isclose(x, y, rtol=1e-2) == False
        # print(x[not_close],y[not_close])
        print(var, x[not_close], y[not_close])


pfcand_phirel [-0.00747986 -0.00318325 -0.00660122  0.01332091  0.03255197] [-0.0078125  -0.00341797 -0.00683594  0.01318359  0.03222656]
pfcand_normchi2 [999. 999. 999. 999. 999. 999. 999. 999. 999. 999.] [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
pfcand_dz [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
pfcand_dzsig [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
pfcand_dxy [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.] [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
pfcand_dxysig [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [62]:
for var in sv_variables:
    x = get_dnn(var)
    y = get_coffea(var)

    if not np.allclose(x, y, rtol=1e-2):
        not_close = np.isclose(x, y, rtol=1e-2) == False
        # print(x[not_close],y[not_close])
        print(var, x[not_close], y[not_close])


In [25]:
(events_pf.FatJet.pt * (1.-events_pf.FatJet.rawFactor))[1]

<Array [465, 251] type='2 * float32'>

In [26]:
events_dnn[0].pf

<Record ... 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0]} type='{"Delta_gen_pt": float32, "e...'>

In [87]:
np.where(np.isnan(pad_val(events_pf.FatJetPFCands.btagSip3dSig, 128)))

(array([  8,  12,  20,  28,  36,  39,  43,  48,  59,  72,  95,  96, 110,
        110, 116, 129, 134, 136, 148, 160, 176, 178, 179, 187, 187, 195,
        209, 223, 235, 247, 270, 272, 291, 294, 296, 300, 315, 330, 332,
        350, 351, 355, 355, 355, 372, 380, 387, 401, 402, 414, 415, 428,
        434, 436, 438, 445, 448, 477, 481, 491, 496]),
 array([  1,   4,  89,  79,  41,  56,  85,  85,  78,  56,  50,  41,  15,
         17,  42,  17,  31,  42,  67,  95,  60,  41, 127,  33,  34,  90,
          0,   9,   0,   0,  51,  85,  31, 110,  19,  26,  98,  25,  44,
         71,  32,  23,  25,  26,   0,   1,  42,  70,  78, 124,   0, 113,
         58,  49,  36, 103, 101,  52,  53,   0,   5]))

In [179]:
d0 = events_pf.PFCands.d0

TypeError: only fields may be assigned in-place (by field name)

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.1/src/awkward/highlevel.py#L1060)

In [112]:
events_pf[8].FatJet

<Array [496, 310] type='2 * float32[parameters={"__doc__": "pt"}]'>

In [127]:
pfcands = events_pf.FatJetPFCands[events_pf.FatJetPFCands.jetIdx == 0][8]
pfcands = pfcands[ak.argsort(pfcands.pt, ascending=False)]

In [132]:
pfcands.btagSip3dSig

<Array [nan, -19.9, 0, ... 35.4, 50.4, 0.553] type='19 * float32[parameters={"__...'>

In [107]:
events_pf.FatJetPFCands.btagSip3dSig[events_pf.FatJetPFCands.jetIdx == 0][8]

<Array [0.553, nan, 20.6, -19.9, ... 0, 0, 0] type='19 * float32[parameters={"__...'>

In [88]:
events_pf.FatJetPFCands.btagSip3dSig[8]

<Array [0.553, nan, 20.6, -19.9, ... 0, 0, 0] type='83 * float32[parameters={"__...'>

In [175]:
get_dnn("pfcand_btagSip3dSig", 20)

array([  0.        , -19.905571  ,   0.        ,  20.609594  ,
         0.        ,   0.        ,   0.        ,   1.4226751 ,
        14.562361  ,   0.        ,   0.        ,   0.49658412,
         0.        ,   0.        ,   0.        ,   0.        ,
        35.411736  ,  50.420128  ,   0.5531049 ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.  

In [133]:
get_coffea("pfcand_btagSip3dSig", 7)

array([  0.        , -19.90625   ,   0.        ,  20.609375  ,
         0.        ,   0.        ,   0.        ,   1.4228516 ,
        14.5625    ,   0.        ,   0.        ,   0.49658203,
         0.        ,   0.        ,   0.        ,   0.        ,
        35.40625   ,  50.40625   ,   0.55322266,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.  

In [99]:
events_pf.FatJet.pt[:3]

<Array [[309, 307], [504, 275], [402, 190]] type='3 * var * float32[parameters={...'>

In [136]:
(events_pf.FatJet.pt * (1.-events_pf.FatJet.rawFactor))[8]

<Array [451, 280] type='2 * float32'>

In [137]:
events_pf.FatJet.pt[8]

<Array [496, 310] type='2 * float32[parameters={"__doc__": "pt"}]'>

In [155]:
events_coffea["fj_pt"][13]

526.5

In [162]:
events_dnn["fj_pt"][13]

458.92169189453125

In [171]:
np.isclose(451, np.array(events_dnn["fj_pt"]), rtol=0.001)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [170]:
events_dnn["fj_pt"][np.isclose(451, np.array(events_dnn["fj_pt"]), rtol=0.001)]

<Array [451] type='1 * float32'>

In [176]:
events_dnn["fj_phi"][20]

2.6756279468536377

In [177]:
events_coffea["fj_phi"][7]

2.67578125

In [172]:
get_dnn("pfcand_energy", 20)

array([215.15346   ,  89.79295   ,  84.584045  ,  26.008333  ,
        29.181103  ,  19.63367   ,   9.582095  ,   8.535585  ,
         7.9552684 ,   5.3122315 ,   4.4018445 ,   3.696305  ,
         3.179364  ,   2.2022815 ,   2.0336158 ,   1.7569836 ,
         1.5180031 ,   1.1371361 ,   0.70038134,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.  

In [166]:
get_coffea("pfcand_energy", 7)

array([215.15256  ,  89.79239  ,  84.58351  ,  26.00817  ,  29.180498 ,
        19.633253 ,   9.582329 ,   8.5357895,   7.9554586,   5.3123584,
         4.401749 ,   3.6963947,   3.1794395,   2.2022614,   2.033571 ,
         1.7570256,   1.5180397,   1.1371614,   0.7003773,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0. 

In [150]:
(events_pf.FatJet.pt * (1.-events_pf.FatJet.rawFactor))[0]

<Array [280, 280] type='2 * float32'>

In [ ]:
pfcands.btagSip3dSig

In [115]:
get_coffea("pfcand_btagSip3dVal", 7)

array([ 6.0429688e+00, -3.5003662e-02,  0.0000000e+00,  7.0190430e-02,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  3.8833618e-03,
        2.1894531e+00,  0.0000000e+00,  0.0000000e+00,  1.8281937e-03,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        1.0703125e+00,  9.4433594e-01,  1.0330200e-02,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
      

In [114]:
events_coffea["pfcand_btagSip3dVal"][7]

array([ 1.0330200e-02,  6.0429688e+00,  7.0190430e-02, -3.5003662e-02,
        0.0000000e+00,  0.0000000e+00,  2.1894531e+00,  3.8833618e-03,
        1.8281937e-03,  9.4433594e-01,  1.0703125e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
      